<a href="https://colab.research.google.com/github/zuzannazak/MgrSarcasm/blob/master/mgr_sie%C4%87_neuronowa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import json
import tensorflow as tf
import requests
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LeakyReLU
import csv

#Sources that i don't want to keep looking for:
#  https://towardsdatascience.com/tensorflow-sarcasm-detection-in-20-mins-b549311b9e91
#  https://medium.com/analytics-vidhya/sarcasm-detection-with-neural-networks-1509578bb17b 
#  https://towardsdatascience.com/7-popular-activation-functions-you-should-know-in-deep-learning-and-how-to-use-them-with-keras-and-27b4d838dfe6
#  https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [23]:
sentences = []
labels = []

with open('/content/drive/MyDrive/STUDIA/train-balanced.csv', 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        labels.append(row[0])
        sentences.append(row[1])

In [3]:
print(pd.DataFrame({'sentence' : sentences[0:10], 'label':labels[0:10]}))

                                            sentence label
0                                   Dang dog, thanks     0
1  to summon the powers of the flying spaghetti m...     0
2       i did that 3rd last 1 by accident last night     0
3  He's insane, used him in DC, better than Blake...     0
4  Forgot about him, he's a pretty pointless card...     0
5                                                hey     0
6                                              yeah?     0
7                                               okay     0
8                                      Condensation?     0
9                                What type of juice?     0


In [24]:
# Splitting the dataset into Train and Test
from sklearn.model_selection import train_test_split
training_size = round(len(sentences) * .75)
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.30, random_state=42)

In [25]:
# Setting tokenizer properties
vocab_size = 50000
oov_tok = "<oov>"

In [26]:
# Fit the tokenizer on Training data
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [27]:
# Setting the padding properties
max_length = 200
trunc_type='post'
padding_type='post'

In [28]:
# Creating padded sequences from train and test data
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [29]:
# Setting the model parameters
embedding_dim = 16
leaky_relu = LeakyReLU(alpha=0.01)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(50, activation=leaky_relu),
    #tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 16)           800000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                850       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 800,901
Trainable params: 800,901
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Setting the model parameters 2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten
embedding_dim=16
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 16)           800000    
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                102432    
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [31]:
# # Training the model
num_epochs = 40
history = model.fit(np.array(training_padded), np.array(training_labels, dtype='int32'), epochs=num_epochs, validation_data=(np.array(testing_padded), np.array(testing_labels, dtype='int32')), verbose=2)




Epoch 1/40
26384/26384 - 282s - loss: 0.6041 - accuracy: 0.6693 - val_loss: 0.5735 - val_accuracy: 0.6990
Epoch 2/40


KeyboardInterrupt: ignored

In [14]:
type(training_padded)

numpy.ndarray

In [ ]:
sentence = ["Yes, definitely",
            "I like cats."]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

NameError: ignored

https://medium.com/analytics-vidhya/sarcasm-detection-with-neural-networks-1509578bb17b

In [19]:
vocab_size=10000
embedding_dim=16
max_length=32
trunc_type='post'
padding_type='post'
oov_tok='<oov>'
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer= Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index
training_sequences=tokenizer.texts_to_sequences(X_train)
training_padded=pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences=tokenizer.texts_to_sequences(X_test)
testing_padded=pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 16)           800000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                102432    
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                

In [34]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history = model.fit(np.array(training_padded), np.array(training_labels, dtype='int32'), batch_size=256, epochs=100, validation_data=(np.array(testing_padded), np.array(testing_labels, dtype='int32')), verbose=1, callbacks=[early_stop])

Epoch 1/100
3298/3298 [==============================] - 65s 19ms/step - loss: 0.6816 - accuracy: 0.5492 - val_loss: 0.5968 - val_accuracy: 0.6901
Epoch 2/100
3298/3298 [==============================] - 63s 19ms/step - loss: 0.6137 - accuracy: 0.6785 - val_loss: 0.5686 - val_accuracy: 0.7110
Epoch 3/100
3298/3298 [==============================] - 63s 19ms/step - loss: 0.5882 - accuracy: 0.7048 - val_loss: 0.5565 - val_accuracy: 0.7134
Epoch 4/100
3298/3298 [==============================] - 64s 19ms/step - loss: 0.5728 - accuracy: 0.7209 - val_loss: 0.5534 - val_accuracy: 0.7166
Epoch 5/100
3298/3298 [==============================] - 63s 19ms/step - loss: 0.5629 - accuracy: 0.7298 - val_loss: 0.5511 - val_accuracy: 0.7172
Epoch 6/100
3298/3298 [==============================] - 63s 19ms/step - loss: 0.5539 - accuracy: 0.7377 - val_loss: 0.5521 - val_accuracy: 0.7164
Epoch 7/100
3298/3298 [==============================] - 62s 19ms/step - loss: 0.5454 - accuracy: 0.7445 - val_loss: 0